In [24]:
# !pip install hsclient
TRIALS = 10
targets = ["beta.hydroshare.org", "www.hydroshare.org"]
clients = {}
filename = "local-build-discover.sh"
diffs= {}

In [25]:
from hsclient import HydroShare
for target in targets:
    hs = HydroShare(host=target, port=443, protocol='https')
    hs.sign_in()
    hs.resources = []
    diffs[target] = []
    clients[target] = hs

In [26]:
from time import time
from functools import wraps
import numpy as np
import scipy
  
def timer_func(func): 
    @wraps(func)
    def wrap_func(*args, **kwargs): 
        t1 = time() 
        result = func(*args, **kwargs)
        t2 = time()
        target = kwargs['target']
        diff = t2-t1
        diffs[target].append(diff)
        print(f'Function {func.__name__!r} executed in {(diff):.4f}s') 
        return result 
    return wrap_func


def create_res(hs, time=True):

    @timer_func
    def create_res_time(hs=None, target=None):
        resource = hs.create()
        hs.resources.append(resource)
    
    if time:
        create_res_time(hs=None, target=None)
    else:
        resource = hs.create()
        hs.resources.append(resource)

@timer_func
def delete_resources(hs=None, target=None):
    for res in hs.resources:
        res.delete()

@timer_func
def download_all_resources(hs=None, target=None):
    for res in hs.resources:
        res.download()

def download_single_resource(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_resource_inner(hs=res, target=None):
        res.download()

    download_single_resource_inner(hs=res, target=target)

def upload_file(hs, target=None):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def upload_file_inner(hs=res, target=None):
        res.file_upload(filename)

    upload_file_inner(hs=res, target=target)


def create_dir_upload_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def create_dir_upload_file_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
    create_dir_upload_file_inner(hs=res, target=target)

def upload_and_movefile(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    
    @timer_func
    def upload_and_movefile_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
        file = res.file(path="New_Folder/20.zip")
        res.file_rename(file, filename)
    upload_and_movefile_inner(hs=res, target=target)

def download_single_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_file_inner(hs=res, target=target):
        file = res.file(path="20.zip")
        res.file_download(file)
    download_single_file_inner(hs=res, target=target)

def delete_single_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_file_inner(hs=res, target=None):
        file = res.file(path="20.zip")
        res.file_delete(file)
    delete_single_file_inner(hs=res, target=target)
        

def run_comparisons(functions_to_run, runs=1):
    for funct in functions_to_run:
        print(f"********* Start {funct.__name__!r} *********")
        for i in range(runs):
            for target in targets:
                client = clients[target]
                print(f"Starting run {i} of {funct.__name__} on {target}...")
                funct(hs=client, target=target)
                print(diffs[target])
        for target in targets:
            client = clients[target]
            print(f"Average time {funct.__name__!r} on {target}: {sum(diffs[target])/len(diffs[target])}")
            print(f"Devation for {funct.__name__!r} on {target}: {np.std(diffs[target])}")
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
        result = scipy.stats.ttest_ind(diffs[targets[0]], diffs[targets[1]])
        print(result)
        print("\n\n")
        for target in targets:
            diffs[target].clear()

def clear_resources():
    for target in targets:
        client = clients[target]
        client.resources = []
        diffs[target].clear()
  

In [29]:
clear_resources()
run_comparisons([download_single_resource, upload_file, create_dir_upload_file, delete_single_file, download_single_file, create_res, upload_and_movefile ], runs=TRIALS)


********* Start 'download_single_resource' *********
Starting run 0 of download_single_resource on beta.hydroshare.org...


In [ ]:
run_comparisons([download_all_resources, delete_resources], runs=1)